In [1]:
from bokeh.io import output_notebook, show, save
from bokeh.models import ColumnDataSource, NumeralTickFormatter
from bokeh.plotting import figure


import pandas as pd

In [2]:
output_notebook()

Loading BokehJS ...

# Dane

In [3]:
artists_word_count = pd.read_csv('data/artists_word_count_agg.csv')
dt_artists_agg_data = pd.read_csv('data/dt_artists_agg_data.csv')

dt_album_agg_data = pd.read_csv('data/dt_album_agg_data.csv')

artists_metadata = pd.read_csv('data/artists_metadata.csv')
artists_metadata = artists_metadata.set_index('artist')

albums_metadata = pd.read_csv('data/albums_metadata.csv')
albums_metadata = albums_metadata.set_index('album_id')

dt_album_agg_data_aug = dt_album_agg_data.drop(columns='artist')
dt_album_agg_data_aug = dt_album_agg_data_aug.join(albums_metadata, on='album_id')

In [4]:
dt_album_agg_data_aug.head()

,album_id,N_songs,N_songs_wL,N_words,N_words_per_s,Avg_in_dict,Avg_in_NAWL,Avg_vulg,Avg_max_N_Anger,Avg_max_N_Fear,...,Avg_min_sum_Happiness,Avg_min_sum_Sadness,Avg_min_sum_Disgust,artist_id,artist,album_name,album_label,album_popularity,album_release_date,album_images.url
0,0Qe1x6jUzZMknaglDsz69T,13,13,2768,0.925009,0.969292,0.208815,0.000000,0.031142,0.015571,...,2.642682,1.623381,1.242428,1PHz9tBsAlizVFqzFVSmlk,Afro Kolektyw,46 Minut Sodomy,Universal Music Polska Sp. z o.o.,13,2014-01-01,https://i.scdn.co/image/ab67616d0000b2731fd1c5...
1,7iZ2fgKBeoR9IW9f0Dolr0,12,3,827,0.724101,0.980653,0.218863,0.000000,0.011050,0.022099,...,2.267523,1.698195,1.248957,1PHz9tBsAlizVFqzFVSmlk,Afro Kolektyw,Piosenki Po Polsku,Universal Music Polska Sp. z o.o.,18,2012-01-01,https://i.scdn.co/image/ab67616d0000b273068c4d...
2,3Y1u8wVxI0hOTbZupN4jvp,14,10,3706,1.510419,0.964382,0.213708,0.004317,0.006313,0.022727,...,2.854107,1.497416,1.205987,1PHz9tBsAlizVFqzFVSmlk,Afro Kolektyw,Połącz kropki,Wytwórnia Krajowa,14,2008-11-10,https://i.scdn.co/image/ab67616d0000b273b5f9e0...
3,2dURAH3fhLP9D7wQL1uWI8,15,10,4379,1.510817,0.952957,0.201644,0.006851,0.011325,0.023783,...,2.386433,1.422059,1.184561,1PHz9tBsAlizVFqzFVSmlk,Afro Kolektyw,Czarno widzę,Wytwórnia Krajowa,16,2006-02-20,https://i.scdn.co/image/ab67616d0000b2731eb6b2...
4,41OLJSiOmkSANMUWfOUOCh,26,11,3775,1.559653,0.932450,0.175364,0.005563,0.015106,0.031722,...,2.592494,1.600950,1.406577,1PHz9tBsAlizVFqzFVSmlk,Afro Kolektyw,Płyta pilśniowa,Wytwórnia Krajowa,21,2001-12-10,https://i.scdn.co/image/ab67616d0000b273c7a1c9...


# Albumy 2D

In [30]:
source = ColumnDataSource(dt_album_agg_data_aug.query('N_songs_wL >= 5').rename(columns={'album_images.url': 'url'}))

TOOLTIPS = """
    <div>
        <div>
            <img
                src="@url" height="200" width="200"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@artist</span>
        </div>
        <div>
            <span style="font-size: 15px; font-weight: bold;">@album_name</span>
        </div>
        <div>
            <span style="font-size: 14px;">@album_label</span>
        </div>
        <div>
            <span style="font-size: 14px;">@album_release_date</span>
        </div>
        <div>
            <span style="font-size: 14px;">@N_words_per_s słów/sek.</span>
        </div>
        <div>
            <span style="font-size: 14px;">@Avg_vulg</span>
        </div>
    </div>
"""

p = figure(plot_width=800, plot_height=800, tooltips=TOOLTIPS,
           title="Liczba słów na sekundę vs. udział wulgaryzmów", margin = [100, 0, 100, 0])

p.circle('N_words_per_s', 'Avg_vulg', size=10, source=source)

p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

p.title.text_font_size = "16pt"


show(p)